# Stage 3.2: Processed Combined Text Data EDA

Exploratory data analysis of the model-ready features after Stage 3 processing.

**Input**: `data/model_ready/{train,test}.csv` (from Stage 3)
**Features**: 186 NLP and text features for predicting review quality

This notebook contains:
* EDA of processed text data with Dataprep.eda and plotting libraries

## Imports and Global Settings

In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# EDA libraries
from dataprep.eda import plot, plot_correlation

# Add project root to path for imports
sys.path.insert(0, str(Path.cwd().parent))
from src.config import PathConfig

pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 200)

## Loading Data

In [ ]:
# Load from model-ready directory (Stage 3 output)
model_ready_dir = PathConfig.get_model_ready_dir()
print(f"Reading from: {model_ready_dir}")

In [ ]:
# For large datasets, limit records loaded
# Set to None to load all records
train_records_to_load = None  # Load all test data (7000 rows)
test_records_to_load = None   # Load all test data (1500 rows)

In [ ]:
# Datatypes are auto-inferred from CSV
# Stage 3 script already applied memory-efficient dtypes
# No manual dtype mapping needed

In [ ]:
# Load data from CSV
train = pd.read_csv(model_ready_dir / "train.csv", nrows=train_records_to_load)
test = pd.read_csv(model_ready_dir / "test.csv", nrows=test_records_to_load)
print(f"Train: {len(train):,} rows, {len(train.columns)} columns")
print(f"Test: {len(test):,} rows, {len(test.columns)} columns")

In [ ]:
# Separate features and targets
# Target columns start with T (T1_REG, T2_CLS, etc.)
target_cols = [c for c in train.columns if c.startswith('T')]
id_cols = ['review_id']
feature_cols = [c for c in train.columns if c not in target_cols + id_cols]

X_train = train[feature_cols]
X_test = test[feature_cols]
y_train = train['T2_CLS_ufc_>0']  # Primary classification target
y_test = test['T2_CLS_ufc_>0']

print(f"Features: {len(feature_cols)}")
print(f"Targets: {target_cols}")

## Basic Overview

In [7]:
train.columns.values

array(['review_id', 'target_clf', 'target_reg', 'review_stars', 'NB_prob',
       'svm_pred', 'grade_level', 'polarity', 'subjectivity', 'word_cnt',
       'character_cnt', 'num_cnt', 'uppercase_cnt', '#@_cnt',
       'sentence_cnt', 'lexicon_cnt', 'syllable_cnt', 'avg_word_len',
       'token_cnt', 'stopword_cnt', 'stopword_pct', 'ent_cnt', 'ent_pct',
       'pos_adj_pct', 'pos_adj_cnt', 'pos_adp_pct', 'pos_adp_cnt',
       'pos_adv_pct', 'pos_adv_cnt', 'pos_aux_pct', 'pos_aux_cnt',
       'pos_conj_pct', 'pos_conj_cnt', 'pos_det_pct', 'pos_det_cnt',
       'pos_intj_pct', 'pos_intj_cnt', 'pos_noun_pct', 'pos_noun_cnt',
       'pos_num_pct', 'pos_num_cnt', 'pos_part_pct', 'pos_part_cnt',
       'pos_pron_pct', 'pos_pron_cnt', 'pos_propn_pct', 'pos_propn_cnt',
       'pos_punct_pct', 'pos_punct_cnt', 'pos_sconj_pct', 'pos_sconj_cnt',
       'pos_sym_pct', 'pos_sym_cnt', 'pos_verb_pct', 'pos_verb_cnt',
       'pos_x_pct', 'pos_x_cnt', 'dep_root_pct', 'dep_root_cnt',
       'dep_acl_pct'

In [8]:
train.head(5)

,review_id,target_clf,target_reg,review_stars,NB_prob,svm_pred,grade_level,polarity,subjectivity,word_cnt,character_cnt,num_cnt,uppercase_cnt,#@_cnt,sentence_cnt,lexicon_cnt,syllable_cnt,avg_word_len,token_cnt,stopword_cnt,stopword_pct,ent_cnt,ent_pct,pos_adj_pct,pos_adj_cnt,pos_adp_pct,pos_adp_cnt,pos_adv_pct,pos_adv_cnt,pos_aux_pct,pos_aux_cnt,pos_conj_pct,pos_conj_cnt,pos_det_pct,pos_det_cnt,pos_intj_pct,pos_intj_cnt,pos_noun_pct,pos_noun_cnt,pos_num_pct,pos_num_cnt,pos_part_pct,pos_part_cnt,pos_pron_pct,pos_pron_cnt,pos_propn_pct,pos_propn_cnt,pos_punct_pct,pos_punct_cnt,pos_sconj_pct,pos_sconj_cnt,pos_sym_pct,pos_sym_cnt,pos_verb_pct,pos_verb_cnt,pos_x_pct,pos_x_cnt,dep_root_pct,dep_root_cnt,dep_acl_pct,dep_acl_cnt,dep_acomp_pct,dep_acomp_cnt,dep_advcl_pct,dep_advcl_cnt,dep_advmod_pct,dep_advmod_cnt,dep_agent_pct,dep_agent_cnt,dep_amod_pct,dep_amod_cnt,dep_appos_pct,dep_appos_cnt,dep_attr_pct,dep_attr_cnt,dep_aux_pct,dep_aux_cnt,dep_auxpass_pct,dep_auxpass_cnt,dep_case_pct,dep_case_cnt,dep_cc_pct,dep_cc_cnt,dep_ccomp_pct,dep_ccomp_cnt,dep_compound_pct,dep_compound_cnt,dep_conj_pct,dep_conj_cnt,dep_csubj_pct,dep_csubj_cnt,dep_csubjpass_pct,dep_csubjpass_cnt,dep_dative_pct,dep_dative_cnt,dep_dep_pct,dep_dep_cnt,dep_det_pct,dep_det_cnt,dep_dobj_pct,dep_dobj_cnt,dep_expl_pct,dep_expl_cnt,dep_intj_pct,dep_intj_cnt,dep_mark_pct,dep_mark_cnt,dep_meta_pct,dep_meta_cnt,dep_neg_pct,dep_neg_cnt,dep_nmod_pct,dep_nmod_cnt,dep_npadvmod_pct,dep_npadvmod_cnt,dep_nsubj_pct,dep_nsubj_cnt,dep_nsubjpass_pct,dep_nsubjpass_cnt,dep_nummod_pct,dep_nummod_cnt,dep_oprd_pct,dep_oprd_cnt,dep_parataxis_pct,dep_parataxis_cnt,dep_pcomp_pct,dep_pcomp_cnt,dep_pobj_pct,dep_pobj_cnt,dep_poss_pct,dep_poss_cnt,dep_preconj_pct,dep_preconj_cnt,dep_predet_pct,dep_predet_cnt,dep_prep_pct,dep_prep_cnt,dep_prt_pct,dep_prt_cnt,dep_punct_pct,dep_punct_cnt,dep_quantmod_pct,dep_quantmod_cnt,dep_relcl_pct,dep_relcl_cnt,dep_xcomp_pct,dep_xcomp_cnt,ent_cardinal_pct,ent_cardinal_cnt,ent_date_pct,ent_date_cnt,ent_event_pct,ent_event_cnt,ent_fac_pct,ent_fac_cnt,ent_gpe_pct,ent_gpe_cnt,ent_language_pct,ent_language_cnt,ent_law_pct,ent_law_cnt,ent_loc_pct,ent_loc_cnt,ent_money_pct,ent_money_cnt,ent_norp_pct,ent_norp_cnt,ent_ordinal_pct,ent_ordinal_cnt,ent_org_pct,ent_org_cnt,ent_percent_pct,ent_percent_cnt,ent_person_pct,ent_person_cnt,ent_product_pct,ent_product_cnt,ent_quantity_pct,ent_quantity_cnt,ent_time_pct,ent_time_cnt,ent_work_of_art_pct,ent_work_of_art_cnt
0,LMghnfV8h5_CxooL9NuYCg,False,0,1,0.99400,-0.42400,8.40000,-0.22875,0.40000,20,136,0,1,0,2,20,33,5.85000,24,7,0.29167,1,0.04167,0.16667,4,0.04167,1,0.16667,4,0.04167,1,0.00000,0,0.00000,0,0.00000,0,0.20833,5,0.08333,2,0.00000,0,0.04167,1,0.00000,0,0.16667,4,0.04167,1,0.00000,0,0.04167,1,0.00000,0,0.08333,2,0.00000,0,0.04167,1,0.04167,1,0.16667,4,0.00000,0,0.12500,3,0.08333,2,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.04167,1,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.04167,1,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.08333,2,0.00000,0,0.04167,1,0.00000,0,0.00000,0,0.00000,0,0.04167,1,0.00000,0,0.00000,0,0.00000,0,0.04167,1,0.00000,0,0.16667,4,0.00000,0,0.00000,0,0.00000,0,0.04167,1,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.00000,0
1,3ppa5clQkm9Ha5knQHSiSw,True,12,4,0.00000,0.60200,6.90000,0.50325,0.66942,215,1229,0,1,0,14,215,298,4.68982,243,108,0.44444,9,0.03704,0.09053,22,0.09465,23,0.06996,17,0.05761,14,0.00000,0,0.10700,26,0.00000,0,0.18107,44,0.00000,0,0.00823,2,0.05350,13,0.08642,21,0.08642,21,0.01235,3,0.00412,1,0.09053,22,0.00000,0,0.06173,15,0.00412,1,0.02058,5,0.01235,3,0.06584,16,0.00000,0,0.05761,14,0.00412,1,0.02058,5,0.02469,6,0.01235,3,0.00000,0,0.05350,13,0.01235,3,0.07819,19,0.06584,16,0.00000,0,0.00000,0,0.00000,0,0.00000,0,0.09053,22,0.02881,7,0.00000,0,0.00000,0,0.00823,2,0.00000,0,0.00412,1,0.00000,0,0.00000,0

In [9]:
train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 183 columns):
 #    Column               Dtype  
---   ------               -----  
 0    review_id            object 
 1    target_clf           bool   
 2    target_reg           int16  
 3    review_stars         int16  
 4    NB_prob              float32
 5    svm_pred             float32
 6    grade_level          float32
 7    polarity             float32
 8    subjectivity         float32
 9    word_cnt             int16  
 10   character_cnt        int16  
 11   num_cnt              int16  
 12   uppercase_cnt        int16  
 13   #@_cnt               int16  
 14   sentence_cnt         int16  
 15   lexicon_cnt          int16  
 16   syllable_cnt         int16  
 17   avg_word_len         float32
 18   token_cnt            int16  
 19   stopword_cnt         int16  
 20   stopword_pct         float32
 21   ent_cnt              int16  
 22   ent_pct              float32
 23   pos_adj_

In [10]:
train.describe()

,target_reg,review_stars,NB_prob,svm_pred,grade_level,polarity,subjectivity,word_cnt,character_cnt,num_cnt,uppercase_cnt,#@_cnt,sentence_cnt,lexicon_cnt,syllable_cnt,avg_word_len,token_cnt,stopword_cnt,stopword_pct,ent_cnt,ent_pct,pos_adj_pct,pos_adj_cnt,pos_adp_pct,pos_adp_cnt,pos_adv_pct,pos_adv_cnt,pos_aux_pct,pos_aux_cnt,pos_conj_pct,pos_conj_cnt,pos_det_pct,pos_det_cnt,pos_intj_pct,pos_intj_cnt,pos_noun_pct,pos_noun_cnt,pos_num_pct,pos_num_cnt,pos_part_pct,pos_part_cnt,pos_pron_pct,pos_pron_cnt,pos_propn_pct,pos_propn_cnt,pos_punct_pct,pos_punct_cnt,pos_sconj_pct,pos_sconj_cnt,pos_sym_pct,pos_sym_cnt,pos_verb_pct,pos_verb_cnt,pos_x_pct,pos_x_cnt,dep_root_pct,dep_root_cnt,dep_acl_pct,dep_acl_cnt,dep_acomp_pct,dep_acomp_cnt,dep_advcl_pct,dep_advcl_cnt,dep_advmod_pct,dep_advmod_cnt,dep_agent_pct,dep_agent_cnt,dep_amod_pct,dep_amod_cnt,dep_appos_pct,dep_appos_cnt,dep_attr_pct,dep_attr_cnt,dep_aux_pct,dep_aux_cnt,dep_auxpass_pct,dep_auxpass_cnt,dep_case_pct,dep_case_cnt,dep_cc_pct,dep_cc_cnt,dep_ccomp_pct,dep_ccomp_cnt,dep_compound_pct,dep_compound_cnt,dep_conj_pct,dep_conj_cnt,dep_csubj_pct,dep_csubj_cnt,dep_csubjpass_pct,dep_csubjpass_cnt,dep_dative_pct,dep_dative_cnt,dep_dep_pct,dep_dep_cnt,dep_det_pct,dep_det_cnt,dep_dobj_pct,dep_dobj_cnt,dep_expl_pct,dep_expl_cnt,dep_intj_pct,dep_intj_cnt,dep_mark_pct,dep_mark_cnt,dep_meta_pct,dep_meta_cnt,dep_neg_pct,dep_neg_cnt,dep_nmod_pct,dep_nmod_cnt,dep_npadvmod_pct,dep_npadvmod_cnt,dep_nsubj_pct,dep_nsubj_cnt,dep_nsubjpass_pct,dep_nsubjpass_cnt,dep_nummod_pct,dep_nummod_cnt,dep_oprd_pct,dep_oprd_cnt,dep_parataxis_pct,dep_parataxis_cnt,dep_pcomp_pct,dep_pcomp_cnt,dep_pobj_pct,dep_pobj_cnt,dep_poss_pct,dep_poss_cnt,dep_preconj_pct,dep_preconj_cnt,dep_predet_pct,dep_predet_cnt,dep_prep_pct,dep_prep_cnt,dep_prt_pct,dep_prt_cnt,dep_punct_pct,dep_punct_cnt,dep_quantmod_pct,dep_quantmod_cnt,dep_relcl_pct,dep_relcl_cnt,dep_xcomp_pct,dep_xcomp_cnt,ent_cardinal_pct,ent_cardinal_cnt,ent_date_pct,ent_date_cnt,ent_event_pct,ent_event_cnt,ent_fac_pct,ent_fac_cnt,ent_gpe_pct,ent_gpe_cnt,ent_language_pct,ent_language_cnt,ent_law_pct,ent_law_cnt,ent_loc_pct,ent_loc_cnt,ent_money_pct,ent_money_cnt,ent_norp_pct,ent_norp_cnt,ent_ordinal_pct,ent_ordinal_cnt,ent_org_pct,ent_org_cnt,ent_percent_pct,ent_percent_cnt,ent_person_pct,ent_person_cnt,ent_product_pct,ent_product_cnt,ent_quantity_pct,ent_quantity_cnt,ent_time_pct,ent_time_cnt,ent_work_of_art_pct,ent_work_of_art_cnt
count,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.00000,100000.0

In [11]:
train.isnull().sum()

review_id              0
target_clf             0
target_reg             0
review_stars           0
NB_prob                0
svm_pred               0
grade_level            0
polarity               0
subjectivity           0
word_cnt               0
character_cnt          0
num_cnt                0
uppercase_cnt          0
#@_cnt                 0
sentence_cnt           0
lexicon_cnt            0
syllable_cnt           0
avg_word_len           0
token_cnt              0
stopword_cnt           0
stopword_pct           0
ent_cnt                0
ent_pct                0
pos_adj_pct            0
pos_adj_cnt            0
pos_adp_pct            0
pos_adp_cnt            0
pos_adv_pct            0
pos_adv_cnt            0
pos_aux_pct            0
pos_aux_cnt            0
pos_conj_pct           0
pos_conj_cnt           0
pos_det_pct            0
pos_det_cnt            0
pos_intj_pct           0
pos_intj_cnt           0
pos_noun_pct           0
pos_noun_cnt           0
pos_num_pct            0


In [12]:
train.duplicated().sum()

0

## Univariate Analysis and Comparision to Target

In [ ]:
def describe_numeric(df, column_name):
    """Plot histogram and boxplot for a numeric column."""
    fig, ax = plt.subplots(1, 2, figsize=(14, 5))
    ax[0].ticklabel_format(style='plain', axis='both')
    ax[1].ticklabel_format(style='plain', axis='both')
    sns.histplot(df[column_name], kde=True, ax=ax[1])
    sns.boxplot(y=df[column_name], color='lightblue', orient='v', ax=ax[0])
    fig.suptitle(column_name.replace('_', ' ').title(), fontsize=16)
    plt.tight_layout()
    
def compare_feature_target(df, feature_name, target_col='T2_CLS_ufc_>0'):
    """Compare feature distribution across target classes."""
    fig, ax = plt.subplots(1, 2, figsize=(14, 5))
    ax[0].ticklabel_format(style='plain', axis='both')
    ax[1].ticklabel_format(style='plain', axis='both')
    sns.histplot(x=df[feature_name], hue=df[target_col], kde=True, ax=ax[0])
    sns.boxplot(x=df[target_col], y=df[feature_name], ax=ax[1])
    fig.suptitle(f'{feature_name} by Target', fontsize=16)
    plt.tight_layout()

In [14]:
# plot_correlation(train, "target_reg", k=10)

In [15]:
# plot(train)